In [1]:
import pandas as pd

In [2]:
train_raw = pd.read_csv('train.csv')
train_raw.head()

,Id,Product_Info_1,Product_Info_2,Product_Info_3,Product_Info_4,Product_Info_5,Product_Info_6,Product_Info_7,Ins_Age,Ht,...,Medical_Keyword_40,Medical_Keyword_41,Medical_Keyword_42,Medical_Keyword_43,Medical_Keyword_44,Medical_Keyword_45,Medical_Keyword_46,Medical_Keyword_47,Medical_Keyword_48,Response
0,2,1,D3,10,0.076923,2,1,1,0.641791,0.581818,...,0,0,0,0,0,0,0,0,0,8
1,5,1,A1,26,0.076923,2,3,1,0.059701,0.600000,...,0,0,0,0,0,0,0,0,0,4
2,6,1,E1,26,0.076923,2,3,1,0.029851,0.745455,...,0,0,0,0,0,0,0,0,0,8
3,7,1,D4,10,0.487179,2,3,1,0.164179,0.672727,...,0,0,0,0,0,0,0,0,0,8
4,8,1,D2,26,0.230769,2,3,1,0.417910,0.654545,...,0,0,0,0,0,0,0,0,0,8


In [3]:
len(train_raw)

59381

In [4]:
train_raw.dropna(axis=1, thresh=50000, inplace=True)

In [5]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(train_raw.Product_Info_2)
Product_Info_2_le = le.transform(train_raw.Product_Info_2)
Product_Info_2_le1 = pd.DataFrame(Product_Info_2_le, columns=['Product_Info_2_le'])

In [6]:
train_raw = pd.concat([train_raw, Product_Info_2_le1], axis=1)

In [7]:
train_raw.drop('Product_Info_2', inplace=True, axis=1)

In [10]:
train_raw.columns[:10]

Index([u'Id', u'Product_Info_1', u'Product_Info_3', u'Product_Info_4',
       u'Product_Info_5', u'Product_Info_6', u'Product_Info_7', u'Ins_Age',
       u'Ht', u'Wt'],
      dtype='object')

In [17]:
import numpy as np
test_vaiable = np.array(train_raw.BMI)* np.array(train_raw.Ins_Age)
test = pd.DataFrame({'BMI*InsAge': test_vaiable, 'BMI': train_raw.BMI, 'Ins_Age': train_raw.Ins_Age})
test.head()

,BMI,BMI*InsAge,Ins_Age
0,0.323008,0.207304,0.641791
1,0.272288,0.016256,0.059701
2,0.428780,0.012799,0.029851
3,0.352438,0.057863,0.164179
4,0.424046,0.177213,0.417910


In [19]:
train_raw['BMI*InsAge'] = test['BMI*InsAge']

In [22]:
train_raw.drop('Ins_Age', inplace=True, axis=1)

In [23]:
train_raw.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 59381 entries, 0 to 59380
Columns: 117 entries, Id to BMI*InsAge
dtypes: float64(7), int64(110)
memory usage: 53.5 MB


In [24]:
for col in train_raw.columns:
    if train_raw[col].isnull().any():
        print col, train_raw[col].isnull().sum()

Employment_Info_1 19
Employment_Info_4 6779
Medical_History_1 8889


In [25]:
emp_info_1_mean = train_raw.Employment_Info_1.median()
train_raw.Employment_Info_1 = train_raw.Employment_Info_1.fillna(emp_info_1_mean)

In [26]:
emp_info_4_mean = train_raw.Employment_Info_4.median()
train_raw.Employment_Info_4 = train_raw.Employment_Info_4.fillna(emp_info_4_mean)

In [27]:
for col in train_raw.columns:
    if train_raw[col].isnull().any():
        print col, train_raw[col].isnull().sum()

Medical_History_1 8889


In [28]:
med_hist_1_mean = train_raw.Medical_History_1.median()
train_raw.Medical_History_1 = train_raw.Medical_History_1.fillna(med_hist_1_mean)

In [29]:
cols_data = list(train_raw.columns.values)

In [30]:
cols_data.remove('Response')
cols_data.remove('Id')

In [31]:
X = train_raw[cols_data]
y = train_raw['Response']

In [32]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler().fit(X)
X_std = scaler.transform(X)

In [33]:
from sklearn.decomposition import PCA
pca = PCA().fit(X_std)
X_std_pca = pca.transform(X_std)

In [34]:
import numpy as np
import xgboost as xgb
# gbm = xgb.XGBClassifier().fit(X_std_pca, y)
def get_params():
    
    params = {}
    params["objective"] = "reg:linear"     
    params["eta"] = 0.05
    params["min_child_weight"] = 240
    params["subsample"] = 0.9
    params["colsample_bytree"] = 0.67
    params["silent"] = 1
    params["max_depth"] = 6
    plst = list(params.items())

    return plst

# def apply_offset(data, bin_offset, sv, scorer=eval_wrapper):
#     # data has the format of pred=0, offset_pred=1, labels=2 in the first dim
#     data[1, data[0].astype(int)==sv] = data[0, data[0].astype(int)==sv] + bin_offset
#     score = scorer(data[1], data[2])
#     return score
from ml_metrics import quadratic_weighted_kappa

def eval_wrapper(yhat, y):  
    y = np.array(y)
    y = y.astype(int)
    yhat = np.array(yhat)
    yhat = np.clip(np.round(yhat), np.min(y), np.max(y)).astype(int)   
    return quadratic_weighted_kappa(yhat, y)

xgb_num_rounds = 800
num_classes = 8

eta_list = [0.05] * 200 
eta_list = eta_list + [0.02] * 500
eat_list = eta_list + [0.01]*100

xgtrain = xgb.DMatrix(X_std_pca, y)

plst = get_params()
model = xgb.train(plst, xgtrain, xgb_num_rounds, learning_rates=eta_list) 
train_preds = model.predict(xgtrain, ntree_limit=model.best_iteration)
print('Train score is:', eval_wrapper(train_preds, y))


('Train score is:', 0.6525239203839341)


In [ ]:
from sklearn.ensemble import RandomForestClassifier
# from sklearn.naive_bayes import MultinomialNB
# from sklearn.linear_model import LogisticRegression
from sklearn.grid_search import GridSearchCV
from time import time
from operator import itemgetter
# model = MultinomialNB(alpha=5)


# Utility function to report best scores
def report(grid_scores, n_top=3):
    top_scores = sorted(grid_scores, key=itemgetter(1), reverse=True)[:n_top]
    for i, score in enumerate(top_scores):
        print("Model with rank: {0}".format(i + 1))
        print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
              score.mean_validation_score,
              np.std(score.cv_validation_scores)))
        print("Parameters: {0}".format(score.parameters))
        print("")

start = time()
# use a full grid over all parameters
param_grid = {"max_depth": [3, None],
              "max_features": [1, 3, 10],
              "min_samples_split": [1, 3, 10],
              "min_samples_leaf": [1, 3, 10],
              "bootstrap": [True, False],
              "criterion": ["gini", "entropy"]}

# run grid search
clf = RandomForestClassifier(n_estimators=20)
grid_search = GridSearchCV(clf, param_grid=param_grid)
start = time()
grid_search.fit(X_std_pca, y)

print("GridSearchCV took %.2f seconds for %d candidate parameter settings."
      % (time() - start, len(grid_search.grid_scores_)))
report(grid_search.grid_scores_)

In [ ]:
from sklearn.cross_validation import cross_val_score
print cross_val_score(model, X_std_pca, y, cv=5, n_jobs=4, verbose=True)

### Preparing test data

In [43]:
test_raw = pd.read_csv('test.csv')
test_raw.head()

,Id,Product_Info_1,Product_Info_2,Product_Info_3,Product_Info_4,Product_Info_5,Product_Info_6,Product_Info_7,Ins_Age,Ht,...,Medical_Keyword_39,Medical_Keyword_40,Medical_Keyword_41,Medical_Keyword_42,Medical_Keyword_43,Medical_Keyword_44,Medical_Keyword_45,Medical_Keyword_46,Medical_Keyword_47,Medical_Keyword_48
0,1,1,D3,26,0.487179,2,3,1,0.611940,0.781818,...,0,0,0,0,0,0,0,0,0,0
1,3,1,A2,26,0.076923,2,3,1,0.626866,0.727273,...,0,0,0,0,0,0,0,0,0,0
2,4,1,D3,26,0.144667,2,3,1,0.582090,0.709091,...,0,0,0,0,0,0,0,0,0,0
3,9,1,A1,26,0.151709,2,1,1,0.522388,0.654545,...,0,0,0,0,0,0,0,0,1,1
4,12,1,A1,26,0.076923,2,3,1,0.298507,0.672727,...,0,0,0,0,0,0,0,0,0,0


In [44]:
Product_Info_2_le_test = le.transform(test_raw.Product_Info_2)
Product_Info_2_le1_test = pd.DataFrame(Product_Info_2_le_test, columns=['Product_Info_2_le'])
test_raw = pd.concat([test_raw, Product_Info_2_le1_test], axis=1)
test_raw.drop('Product_Info_2', inplace=True, axis=1)

In [45]:
test_raw1 = test_raw[cols_data]
test_raw1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19765 entries, 0 to 19764
Columns: 116 entries, Product_Info_1 to Product_Info_2_le
dtypes: float64(8), int64(108)
memory usage: 17.6 MB


In [46]:
for col in test_raw1.columns:
    if test_raw1[col].isnull().any():
        print col, test_raw1[col].isnull().sum()

Employment_Info_1 3
Employment_Info_4 2137
Medical_History_1 2972


In [47]:
test_raw1.Employment_Info_1 = test_raw1.Employment_Info_1.fillna(emp_info_1_mean)
test_raw1.Employment_Info_4 = test_raw1.Employment_Info_4.fillna(emp_info_4_mean)
test_raw1.Medical_History_1 = test_raw1.Medical_History_1.fillna(med_hist_1_mean)

/home/ubuntu/anaconda2/lib/python2.7/site-packages/pandas/core/generic.py:2387: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [48]:
test_std = scaler.transform(test_raw1)
test_std_pca = pca.transform(test_std)

In [ ]:
# model_fit = model.fit(X_std_pca, y)
# predictions = model.predict(test_std_pca)


In [49]:
xgtest = xgb.DMatrix(test_std_pca)

# plst = get_params()
# model = xgb.train(plst, xgtrain, xgb_num_rounds, learning_rates=eta_list) 
test_preds = model.predict(xgtest, ntree_limit=model.best_iteration)

In [78]:
predictions_df = pd.DataFrame({'Response': test_preds})

In [89]:
def rounding(i):
    if i <1:
        return 1
    elif i >8:
        return 8
    else:
        return round(i)
    
predictions_df.Response = predictions_df.Response.apply(rounding)

In [90]:
predictions_df.Response = predictions_df.Response.apply(lambda x: int(x))

In [91]:
predictions_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19765 entries, 0 to 19764
Data columns (total 1 columns):
Response    19765 non-null int64
dtypes: int64(1)
memory usage: 308.8 KB


In [92]:
predictions_df.head()

,Response
0,2
1,5
2,5
3,4
4,4


In [93]:
predictions_df_submit = pd.concat([test_raw.Id, predictions_df.Response], axis=1)

In [94]:
predictions_df_submit.to_csv('xgbtrial_rounding.csv', index=False)